# Qualitatively Evaluate the net (after applying BN)

> **NOTE:** After running BN it is a good idea to restart the kernel, etc. and make sure that the GPU RAM is not being hogged...

In [ ]:
%pylab notebook
!nvidia-smi

In [ ]:
import caffe

In [ ]:
import os

In [ ]:
# EDIT this based on the results of nvidia-smi; 
#   -- Use the CPU if you do not have GPU available
#   -- If you have multiple GPU's pick a free one
caffe.set_mode_gpu()
caffe.set_device(0)

In [ ]:
# Right now, `download-model-weights.sh` puts the CRF weights in this location.
# This location is not ideal; if it gets changed then clearly you will have to edit this cell.
CRF_WEIGHTS = '../../crf-deploy/test_weights.caffemodel'

In [ ]:
net = caffe.Net('crf_inference_net.prototxt', CRF_WEIGHTS, caffe.TEST)

In [ ]:
EVAL_FILES = [fn.strip() for fn in open('/data/traintest/i12/fold_01/eval.txt')]

In [ ]:
# EVAL_FILES moved on this system
EVAL_FILES = [f.replace('data/training/independant_12_layers', '/data/traintest/i12') for f in EVAL_FILES]

In [ ]:
ls *.caffemodel

In [ ]:
from pyfacades.util import softmax

In [ ]:
FEATURES = ['facade', 'window', 'door', 'cornice', 'sill', 'balcony', 'blind', 'deco', 'molding', 'pillar', 'shop']
R1_WEIGHTS = 'test_weights_from_peihao.caffemodel' #'deploy/test_weights.caffemodel'
R1_LAYOUT = 'modified-inference-net.prototxt'

NEG = NEGATIVE = 0
POS = POSITIVE = 2
EDG = EDGE = 3

import numpy as np
import caffe
init_net = caffe.Net(R1_LAYOUT, R1_WEIGHTS, caffe.TEST)

# Set the batch size to one
init_net.blobs['data'].reshape(1, 3, 512, 512)
init_net.reshape()

def priors(im):
    """
    :param im: An input image, shape 3x512x512
    :type im: np.ndarray
    """
    result = init_net.forward(data=np.array([im[:3]]))
    probs = [init_net.blobs['prob-{}'.format(feature)].data[0,(NEG,POS,EDG)] for feature in FEATURES]
    probs = np.concatenate(probs, axis=0)
    return probs

def iterate(net):
    probs = [softmax(net.blobs['conv-{}'.format(feature)].data[0,(0,2,3)]) for feature in FEATURES]
    #probs = [net.blobs['prob-{}'.format(feature)].data[0,(NEG,POS,EDG)] for feature in FEATURES]
    probs = np.concatenate(probs, axis=0)
    net.blobs['data'].data[0,3:] = probs
    result = net.forward(data=net.blobs['data'].data)
    return result
    
def prepare(im, prior=None):
    if prior is None:
        prior = priors(im)
    assert prior.shape == (len(FEATURES)*3, 512, 512)
    concat = np.concatenate([im[:3], prior])
    return np.array([concat])

In [ ]:
def plot_iter(fig=None):
    fig = fig or figure(figsize=(10,4))
    subplot(131)
    imshow(net.blobs['data'].data[0,:3].transpose(1,2,0)/255.)
    axis('off')
    subplot(132)
    imshow(net.blobs['data'].data[0,:3].transpose(1,2,0)/255.)
    imshow(net.blobs['prob-window'].data[0,2], alpha=0.65)
    axis('off')
    subplot(133)
    imshow(net.blobs['prob-window'].data[0,2])
    axis('off')
    tight_layout()
    fig.canvas.draw()

In [ ]:
idx = 24
data_with_targets = np.load(EVAL_FILES[idx])
data, targets = data_with_targets[:3], data_with_targets[3:]

In [ ]:
dirname = 'frame-image-{}'.format(idx)
!mkdir -p {dirname}

results = net.forward(data=prepare(data))
fig = figure(figsize=(10,4))
for i in range(1000):
    iterate(net)
    title('Iteratin {}'.format(i))
    plot_iter(fig)
    savefig(os.path.join(dirname, 'frame_{:05}'.format(i+1)))

In [ ]:
!ffmpeg -i frame-image-{idx}/frame_%05d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p  video-image-{idx}.mp4

[This is the video](video-image-24.mp4)